
##Descripción del Proceso


Para almacenar los archivos en una cuenta de almacenamiento de Azure, puedes utilizar la biblioteca azure-storage-blob de Python. Aquí te dejo el código modificado que descarga los archivos y los almacena en Azure Blob Storage.

Primero, asegúrate de tener instalada la biblioteca azure-storage-blob:

1. Instalación de Librerías: Se instalan las librerías necesarias para interactuar con Azure Blob Storage, realizar solicitudes HTTP y analizar contenido HTML.

2. Chequeo de Conectividad: Se define una función para verificar la conectividad con la URL del repositorio.

3. Configuración de la API: Se configuran las credenciales y la URL base para la búsqueda de documentos en la API del repositorio.

4. Configuración de Azure Blob Storage: Se configuran las credenciales y el cliente de Azure Blob Storage para la subida de archivos.

5. Realización de Búsqueda: Se realiza una búsqueda en la API del repositorio con la consulta "Cacao".

6. Exploración de Resultados: Se exploran los resultados de la búsqueda y se intentan descargar los archivos PDF relacionados con la temática de "Cacao".

7. Descarga y Subida a Azure Blob Storage: Se descargan los archivos PDF encontrados y se suben a Azure Blob Storage.

Este Notebook está diseñado para automatizar el proceso de búsqueda, descarga y almacenamiento de archivos relevantes desde un repositorio web hacia Azure Blob Storage

In [ ]:
!pip install azure-storage-blob
!pip install requests
!pip install beautifulsoup4 requests

## 1. Chequeo de conectividad

In [ ]:
import requests

def check_connectivity(url):
  try:
    response = requests.get(url, timeout=5)  # Set a timeout to avoid waiting too long
    response.raise_for_status()  # Raise an exception for bad status codes
    print(f"{url} is reachable!")
  except requests.exceptions.RequestException as e:
    print(f"{url} is not reachable. Error: {e}")

check_connectivity('http://repository.agrosavia.co')

http://repository.agrosavia.co is reachable!


## 2. Variables para crear el contener


Este código hace lo siguiente:

Realiza una búsqueda en la API y obtiene los resultados.
Genera las URLs del repositorio a partir de los resultados de la búsqueda.
Para cada URL del repositorio:
Hace una solicitud GET para obtener el contenido HTML.
Parsear el HTML utilizando BeautifulSoup para encontrar enlaces a archivos.
Descarga los archivos encontrados.
Sube los archivos descargados a Azure Blob Storage usando las credenciales SAS proporcionadas.
Asegúrate de ajustar cualquier detalle según tus necesidades específicas.


*   Normalización del Nombre del Contenedor: El nombre del contenedor se normaliza eliminando caracteres no alfanuméricos y convirtiéndolo a minúsculas.

*   Creación del Contenedor y Estructura de Carpetas: Se crea el contenedor y se añade la estructura de carpetas correspondiente.

*   Configuración de Azure Blob Storage: Se utiliza el BlobServiceClient para conectarse a Azure Blob Storage con la URL y el token SAS proporcionados.

*   Subida de Archivos a las Carpetas Correspondientes: Los archivos se suben a las carpetas correspondientes dentro del contenedor basado en su tipo.

*   Subida del Archivo CSV: Se sube el archivo CSV de metadatos al contenedor.

Este código permite que los archivos se almacenan en Azure Blob Storage en las ubicaciones correctas, y también sube un archivo CSV con los metadatos al mismo contenedor.

## 3.Configuración y búsqueda en la API

## 3. Exploración y descarga de archivos


Uso de un Conjunto para Archivos Subidos: Se utiliza un conjunto uploaded_files para rastrear los nombres de los archivos que ya han sido subidos, evitando así la duplicación.
Verificación de Duplicados: Antes de descargar y subir un archivo, se verifica si el archivo ya ha sido procesado verificando su nombre en el conjunto uploaded_files.

De otra parte: Función extract_metadata: Esta función se asegura de extraer los metadatos de manera robusta, manejando diferentes posibles claves en la respuesta.
Corrección de Claves de Metadatos: Se actualizó la función de extracción de metadatos para buscar diferentes posibles nombres de claves (btitle y title, date y creationdate).

In [ ]:
import requests
from bs4 import BeautifulSoup
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import os
import pandas as pd
from datetime import datetime
import re

# Configuración de la API
API_KEY = 'l8xx066955af8e434edb84fc0fe545a1c1c6'
BASE_URL = 'https://api-na.hosted.exlibrisgroup.com/primo/v1/search'
QUERY = 'Cacao'
REPOSITORY_BASE_URL = 'https://repository.agrosavia.co'

# Normalizar QUERY y obtener la fecha y hora actuales
normalized_query = re.sub(r'[^a-zA-Z0-9]', '', QUERY.lower())
timestamp = datetime.now()
date_str = timestamp.strftime('%Y%m%d')
hour_str = timestamp.strftime('%H%M%S')

# Nombre del contenedor
container_name = f"{normalized_query}{date_str}{hour_str}"

# Configuración de Azure Blob Storage
account_url = "https://storageyocampo.blob.core.windows.net"
credential = "sv=2022-11-02&ss=b&srt=sco&sp=rwdlaciytfx&se=2025-01-01T01:34:15Z&st=2024-07-14T17:34:15Z&spr=https,http&sig=XXXXXXXXXXXXXXXXXX3D"
blob_service_client = BlobServiceClient(account_url=account_url, credential=credential)
container_client = blob_service_client.create_container(container_name)

# Crear estructura de carpetas dentro del contenedor
folder_structure = ["pdf", "excel", "word", "video", "other"]
for folder in folder_structure:
    blob_client = container_client.get_blob_client(f"{folder}/")
    blob_client.upload_blob('', overwrite=True)  # Crear carpeta vacía

# Función para extraer metadatos de forma segura
def extract_metadata(addata):
    return {
        'title': addata.get('btitle', [None])[0] if 'btitle' in addata else addata.get('title', [None])[0],
        'creator': addata.get('creator', [None])[0],
        'creationdate': addata.get('date', [None])[0] if 'date' in addata else addata.get('creationdate', [None])[0],
        'publisher': addata.get('publisher', [None])[0],
    }

# Función para determinar el tipo de archivo basado en la extensión y obtener la ubicación
def get_file_location_and_type(filename):
    extension = filename.split('.')[-1].lower()
    if extension == 'pdf':
        return 'pdf', 'pdf'
    elif extension in ['doc', 'docx']:
        return 'word', 'word'
    elif extension in ['xls', 'xlsx']:
        return 'excel', 'excel'
    elif extension in ['mp4', 'avi', 'mov', 'flv', 'wmv']:
        return 'video', 'video'
    else:
        return 'other', 'other'

# Lista para almacenar metadatos
metadata_list = []

# Obtener el número total de elementos
params_total = {
    'vid': '57BAC_INST:BAC',
    'scope': 'RI_BAC',
    'q': f'any,contains,{QUERY}',
    'apikey': API_KEY,
    'limit': 1,
    'offset': 0,
    'sort': 'date'
}

response_total = requests.get(BASE_URL, params=params_total)
data_total = response_total.json()
total_items = data_total.get('info', {}).get('total', 0)

# Configurar la paginación
page_size = 50  # Número de elementos por página
pages = (total_items // page_size) + 1

# Explorar y descargar archivos con metadatos
uploaded_files = set()

for page in range(pages):
    params = {
        'vid': '57BAC_INST:BAC',
        'scope': 'RI_BAC',
        'q': f'any,contains,{QUERY}',
        'apikey': API_KEY,
        'limit': page_size,
        'offset': page * page_size,
        'sort': 'date'
    }

    response = requests.get(BASE_URL, params=params)
    data = response.json()

    for item in data.get('docs', []):
        pdf_name = None
        metadata = {}

        if 'pnx' in item and 'addata' in item['pnx']:
            addata = item['pnx']['addata']
            if 'url' in addata and addata['url']:
                pdf_name = addata['url'][0].split('/')[-1]
            metadata = extract_metadata(addata)
        if not pdf_name and 'delivery' in item:
            delivery = item['delivery']
            if 'electronicServices' in delivery and delivery['electronicServices']:
                pdf_name = delivery['electronicServices'][0]['serviceUrl'].split('/')[-1]
            elif 'availabilityLinksUrl' in delivery and delivery['availabilityLinksUrl']:
                pdf_name = delivery['availabilityLinksUrl'][0].split('/')[-1]

        if pdf_name:
            repository_url = f"{REPOSITORY_BASE_URL}/handle/20.500.12324/{pdf_name}"
            print(f"Exploring {repository_url}")

            page_response = requests.get(repository_url)
            if page_response.status_code == 200:
                soup = BeautifulSoup(page_response.content, 'html.parser')
                for a_tag in soup.find_all('a', href=True):
                    if '/bitstream/handle/' in a_tag['href']:
                        file_url = REPOSITORY_BASE_URL + a_tag['href']
                        file_name = file_url.split('/')[-1].split('?')[0]

                        if file_name not in uploaded_files:
                            uploaded_files.add(file_name)

                            # Descargar el archivo
                            file_response = requests.get(file_url)
                            if file_response.status_code == 200:
                                # Determinar la ubicación y tipo de archivo
                                location, file_type = get_file_location_and_type(file_name)
                                blob_client = container_client.get_blob_client(f"{location}/{file_name}")
                                blob_client.upload_blob(file_response.content, overwrite=True)
                                print(f"Uploaded {file_name} to Azure Blob Storage in {location} with metadata: {metadata}")
                            else:
                                print(f"Failed to download {file_url}")

                            # Actualizar metadatos con el nombre y tipo de archivo
                            metadata.update({
                                'idfile': pdf_name,
                                'filename': file_name,
                                'typefile': file_type
                            })

                            # Agregar metadatos a la lista
                            metadata_list.append(metadata)

                            # Imprimir metadatos
                            print(f"Metadata for {pdf_name}: {metadata}")

            else:
                print(f"Failed to access {repository_url}")
        else:
            print(f"No 'url' found in item: {item}")

# Guardar metadatos en un archivo CSV
csv_filename = f"carga_{container_name}.csv"
metadata_df = pd.DataFrame(metadata_list)
metadata_df.to_csv(csv_filename, index=False)

# Subir el archivo CSV al contenedor
blob_client = container_client.get_blob_client(f"{csv_filename}")
with open(csv_filename, "rb") as data:
    blob_client.upload_blob(data, overwrite=True)

print(f"Metadata saved to {csv_filename} and uploaded to Azure Blob Storage")


Exploring https://repository.agrosavia.co/handle/20.500.12324/33623
Uploaded Ver_video_33623.mp4 to Azure Blob Storage in video with metadata: {'title': 'Cacao', 'creator': None, 'creationdate': '2013', 'publisher': None}
Metadata for 33623: {'title': 'Cacao', 'creator': None, 'creationdate': '2013', 'publisher': None, 'idfile': '33623', 'filename': 'Ver_video_33623.mp4', 'typefile': 'video'}
Exploring https://repository.agrosavia.co/handle/20.500.12324/29864
Uploaded 27319_15935.pdf to Azure Blob Storage in pdf with metadata: {'title': 'La poda en cacao.', 'creator': None, 'creationdate': '1988', 'publisher': None}
Metadata for 29864: {'title': 'La poda en cacao.', 'creator': None, 'creationdate': '1988', 'publisher': None, 'idfile': '29864', 'filename': '27319_15935.pdf', 'typefile': 'pdf'}
Exploring https://repository.agrosavia.co/handle/20.500.12324/29867
Uploaded 27323_15939.pdf to Azure Blob Storage in pdf with metadata: {'title': 'El beneficio del cacao en grano.', 'creator': No

In [ ]:
print("Total de elementos de la consulta: " + str(total_items))

Total de elementos de la consulta: 1047


## 5. Identifica video

Configuración de Azure Blob Storage: Utiliza el account_url y credential proporcionados para configurar el cliente del servicio de blob.

Nombre del Contenedor: Se utiliza la misma lógica de generación de nombre de contenedor que en el código principal.

Listar Blobs: Se lista el contenido de la carpeta de video (video/) dentro del contenedor especificado.

Impresión de Archivos: Imprime el nombre de cada blob encontrado en la carpeta de video.

Este fragmento de código te permitirá verificar los archivos que se han almacenado en la carpeta de video dentro del contenedor de Azure Blob Storage.

In [ ]:
from azure.storage.blob import BlobServiceClient

# Configuración de Azure Blob Storage
account_url = "https://storageyocampo.blob.core.windows.net"
credential = "sv=2022-11-02&ss=b&srt=sco&sp=rwdlaciytfx&se=2025-01-01T01:34:15Z&st=2024-07-14T17:34:15Z&spr=https,http&sig=bM3SKXXXXXXXXXXXXXXXXX3D"
#***** se especifica el nombre del contenedor objetivo
# Quitar el comentario si se trata de otro Blob Container obtenido en otra ejecución.
container_name ="cacao20240714183705"

folder_name = "video/"


# Crear el cliente de servicio de blob
blob_service_client = BlobServiceClient(account_url=account_url, credential=credential)
container_client = blob_service_client.get_container_client(container_name)

# Listar archivos en la carpeta de video
print(f"Archivos en la ruta {container_name}/{folder_name}:")
blob_list = container_client.list_blobs(name_starts_with=folder_name)
for blob in blob_list:
    print(blob.name)


Archivos en la ruta cacao20240714183705/video/:
video/
video/34666.mp4
video/Modelo%20Productivo%20para%20el%20cultivo%20de%20cacao%20%28Theobroma%20cacao%20L.%29%20para%20el%20departamento%20de%20Santander.mp4
video/Ver_Documento_39523.mp4
video/Ver_Documento_39524.mp4
video/Ver_Video_36375.mp4
video/Ver_documento_39274.mp4
video/Ver_video_33541.mp4
video/Ver_video_33563.mp4
video/Ver_video_33564.mp4
video/Ver_video_33565.mp4
video/Ver_video_33623.mp4
video/Ver_video_33624.mp4
video/Ver_video_33759.mp4
video/Ver_video_36355.mp4
video/Ver_video_36362.mp4
video/Ver_video_36367.mp4
video/Ver_video_38267.mp4
video/Ver_video_38284.mp4
video/Ver_video_38306.mp4
video/Ver_video_38418.mp4
video/Ver_video_38523.mp4
video/Ver_video_38532.mp4
video/Ver_video_38536.mp4
video/Ver_video_39106.mp4
video/Ver_video_39135.mp4
video/Ver_video_39136.mp4
video/Ver_video_39137.mp4
video/Ver_video_39138.mp4
video/Ver_video_39139.mp4
video/Ver_video_39140.mp4
video/Ver_video_39142.mp4
video/Ver_video_39143.m